<a href="https://colab.research.google.com/github/ClauCll/Curso_ML_A_Z/blob/master/Regresi%C3%B3n/Regresion_Bajas_Antiguedad_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Cuantas serán las bajas voluntarias de los clientes fijos para el periodo 202009 - 202012
 

In [1]:
import pandas as pd
import numpy as np
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
import gspread
from oauth2client.client import GoogleCredentials 
from google.colab import auth
# Autoriza Claudia
auth.authenticate_user()
gc = gspread.authorize (GoogleCredentials.get_application_default())

### leemos datos

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
data = pd.read_csv('/content/drive/My Drive/Dataset/BAJAS.txt',delimiter='~')
X  = data.iloc[:,:-1].values
Y  = data.iloc[:,9]
Y = Y[:,np.newaxis]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  after removing the cwd from sys.path.


In [7]:
#Dimensiones
data.shape

(19, 10)

In [9]:
data.TOTAL_BAJAS

0     1961
1     2435
2     1524
3      978
4     1031
5     1548
6     1659
7     1292
8     1458
9     1534
10    1114
11    1201
12    1221
13    1533
14    1084
15    1816
16     817
17    1018
18    1476
Name: TOTAL_BAJAS, dtype: int64

In [10]:
# Variables numéricas
data.select_dtypes(include=['float64', 'int']).describe()

,ANIO_BAJA,MES_BAJA,TRIM_1,TRIM_2,TRIM_3,TRIM_4,ENTRE_1_2_ANIOS,ENTRE_3_4_ANIOS,MAS_5_ANIOS,TOTAL_BAJAS
count,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000
mean,2019.368421,5.578947,42.789474,24.368421,22.789474,206.052632,647.736842,426.421053,35.105263,1405.263158
std,0.495595,3.321910,22.069534,10.430780,9.186820,79.663510,149.704465,263.831031,42.276727,392.473473
min,2019.000000,1.000000,19.000000,11.000000,7.000000,98.000000,350.000000,231.000000,0.000000,817.000000
25%,2019.000000,3.000000,27.500000,17.500000,17.500000,147.500000,546.000000,313.000000,5.000000,1099.000000
50%,2019.000000,5.000000,39.000000,21.000000,20.000000,181.000000,658.000000,371.000000,11.000000,1458.000000
75%,2020.000000,7.500000,47.500000,28.500000,26.000000,268.000000,733.500000,430.000000,48.000000,1541.000000
max,2020.000000,12.000000,99.000000,54.000000,45.000000,374.000000,980.000000,1470.000000,139.000000,2435.000000


In [12]:
# Valores observados de 1trim
# ==============================================================================
data.TRIM_1.value_counts()

20    2
43    2
35    2
29    1
26    1
88    1
23    1
54    1
19    1
51    1
44    1
42    1
39    1
69    1
99    1
34    1
Name: TRIM_1, dtype: int64

In [ ]:
# Se convierte la variable MES_BAJA tipo string
# ==============================================================================
#datos.MES_BAJA = datos.MES_BAJA.astype("str")

## Modelo SVM

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVR

In [ ]:
x  = x_mes[ c3[:,0]==1 , 0 ]
y  = Y [ c3[:,0]==1 , 0 ]
x  = x[:,np.newaxis]
y  = y[:,np.newaxis]

x_train, x_test, y_train, y_test = train_test_split(x,y ,test_size=0.2, random_state=0)

In [ ]:
svr= SVR(kernel='rbf',degree=3, C=1.0, epsilon=0.2)
svr.fit(x_train,y_train)

In [ ]:
y_predic = svr.predict(x_test)

In [ ]:
print(svr.score(x_test,y_predic))

1.0
